In [1]:
!git clone https://github.com/guo-research-group/AD-Ray-Optics

fatal: destination path 'AD-Ray-Optics' already exists and is not an empty directory.


In [6]:
%cd /content/AD-Ray-Optics
!python setup.py develop

/content/AD-Ray-Optics
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other s

In [4]:
pip install -r requirements.txt

In [2]:
#Restart run time before running the rest of code
from rayoptics import *
from rayoptics.environment import *

In [4]:
from scipy.optimize import minimize
import numpy as np

In [5]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax.numpy import sqrt, copysign, sin
jax.config.update("jax_enable_x64", True)

In [6]:
isdark = False

In [7]:
def lens(f, zoom_adjust, R2, d_L, wv, n, pupil_r, p1=[0],p2=[0]):
    opm = OpticalModel()
    sm  = opm['seq_model']
    osp = opm['optical_spec']
    pm = opm['parax_model']
    em = opm['ele_model']
    pt = opm['part_tree']
    ar = opm['analysis_results']
    osp['pupil'] = PupilSpec(osp, key=['object', 'pupil'], value=pupil_r*2)
    osp['fov'] = FieldSpec(osp, key=['object', 'angle'], value=2.5, flds=[0., 0.707, 1.], is_relative=True)
    osp['wvls'] = WvlSpec([('F', 0.5), (wv, 1.0), ('C', 0.5)], ref_wl=1)

    opm.radius_mode = True

    sm.gaps[0].thi = 5

    #remaining_height = f

    #sm.add_surface([R1, d_L, n, pupil_r])
    #sm.ifcs[sm.cur_surface].profile = RadialPolynomial(r=R1, coefs=p1)
    #sm.set_stop()

    #remaining_height -= d_L
    #gap = 0
    #remaining_height -= gap


    sm.add_surface([-R2, 1, n, pupil_r])
    #sm.ifcs[sm.cur_surface].profile = RadialPolynomial(r=abs(R2), coefs=p2)
    sm.set_stop()


    sm.add_surface([R2, 1+zoom_adjust, n-0.5, pupil_r])
    sm.set_stop()


    sm.add_surface([R2+3, 0, n+0.5, pupil_r])
    sm.set_stop()

    sm.add_surface([-R2-3, 1, n-0.5, pupil_r])
    sm.set_stop()


    sm.add_surface([-R2, 1, n, pupil_r])
    sm.set_stop()

    sm.add_surface([R2, 1.75, n-0.5, pupil_r])
    sm.set_stop()


    sm.add_surface([-R2-3, 1.7, n, pupil_r])
    sm.set_stop()

    if zoom_adjust > 0:
      sm.add_surface([R2+3, 1, n-0.5, pupil_r])
      sm.set_stop()
    elif zoom_adjust == 0:
      sm.add_surface([R2+3, 1.5, n-0.5, pupil_r])
      sm.set_stop()
    else:
      sm.add_surface([R2+3, 1.6, n-0.5, pupil_r])
      sm.set_stop()

    #remaining_height -= d_L


    #sm.add_surface([R1, remaining_height ])

    #sm.gaps[-1].thi = remaining_height - d_L
    n=60
    opm.update_model()

    return sm

In [ ]:
from visualize.lens import visualize_lens
from visualize.rays import visualize_rays
import plotly.graph_objects as go

wv = 400.5618
pupil_r = 1.5
n = 1.5
# Do not adjust above values

R2 = -8 # Radius of the surfaces
f = 15 # How far the rays travel
d_L = 1.5 # distance between lens's thickness
zoom_adjust = -0.25 # Choose either -0.25 or 0 or 0.5

sm = lens(f, zoom_adjust, R2, d_L, wv, n, pupil_r)
ray_data = visualize_rays(sm, x_offsets = np.linspace(-1,1,5), y_offsets = np.linspace(-1,1,5))
lens_data = visualize_lens(sm)
data = []
data.extend(ray_data)
data.extend(lens_data)
fig = go.Figure(data = data)
fig.show()

: 